# Introduction

- Webscrape https://www.tennislive.net for pro-level match data
- Output two .csv files 
    - Single row dataframe for summary statistics dashboard
    - point by point (Pbp) level csv file for Match Viewer website => to attach timestamps using website tagger

need to write some documentation so

1. People reviewing your GitHub can understand what’s it’s doing. (Make a read me file)

2. We can use this in our products for tennis consulting

# Install Packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [2]:
page = requests.get('https://www.tennislive.net/atp/match/bernard-tomic-VS-govind-nanda/m25-tulsa-2024/')

In [3]:
soup = BeautifulSoup(page.text, 'html')

### Check Status Code
- 200 = success
- 404 = failed

In [ ]:
page.status_code

# Single Row For Summary Statistics Dashboard

### Match Info

In [5]:
table = soup.find('table', class_ = 'table_pmatches')

date = soup.find('td', class_ = 'w50').text.strip().split(' ')[0]
round_info = soup.find_all('td', class_='w50')[1].text.strip()
player1 = soup.find_all('td', class_='w130')[0].text.strip()
player2 = soup.find_all('td', class_='w130')[1].text.strip()
score = soup.find('span', id='score').text.strip()
tournament = soup.find('td', class_='w200').find('a').text.strip()

# Create a DataFrame
data = {
    'Date': [date],
    'Round': [round_info],
    'Player 1': [player1],
    'Player 2': [player2],
    'Score': [score],
    'Tournament': [tournament]
}

df_match_info = pd.DataFrame(data)

In [ ]:
df_match_info

### Match Statistics

In [7]:
# Function to extract data for a given statistic
def extract_statistic(statistic):
    row = soup.find('td', string=statistic).parent
    player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
    player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
    return player1_stat, player2_stat

statistics = [
    '1st SERVE %',
    '1st SERVE POINTS WON',
    '2nd SERVE POINTS WON',
    'TOTAL RETURN POINTS WON',
    'TOTAL POINTS WON',
    'DOUBLE FAULTS',
    'ACES'
]

# List of statistics to extract 
stats_length = len(soup.find_all('td', class_='info_txt'))

if stats_length == 8:
    statistics.insert(3, 'BREAK POINTS WON')  # Insert 'BREAK POINTS WON' at the correct position

# Dictionary to hold the statistics
data = {}

# Extract and store statistics for both players
for stat in statistics:
    player1_stat, player2_stat = extract_statistic(stat)
    stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
    data[f'{stat_name}_player1'] = player1_stat
    data[f'{stat_name}_player2'] = player2_stat

# Convert the dictionary to a DataFrame
df_stats = pd.DataFrame([data])

In [ ]:
df_stats

### Combine the DataFrames by column binding them


In [ ]:
# Combine the DataFrames by column binding them
df_combined = pd.concat([df_match_info, df_stats], axis=1)

df_combined

In [10]:

# # Extract match information
# date = soup.find('td', class_='w50').text.strip().split(' ')[0]
# round_info = soup.find_all('td', class_='w50')[1].text.strip()
# player1 = soup.find_all('td', class_='w130')[0].text.strip()
# player2 = soup.find_all('td', class_='w130')[1].text.strip()
# score = soup.find('span', id='score').text.strip()
# tournament = soup.find('td', class_='w200').find('a').text.strip()

# # Create a DataFrame for match info
# match_data = {
#     'Date': [date],
#     'Round': [round_info],
#     'Player 1': [player1],
#     'Player 2': [player2],
#     'Score': [score],
#     'Tournament': [tournament]
# }
# df_match_info = pd.DataFrame(match_data)

# # Function to extract data for a given statistic
# def extract_statistic(statistic):
#     row = soup.find('td', string=statistic).parent
#     player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
#     player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
#     return player1_stat, player2_stat

# # List of statistics to extract
# statistics = [
#     '1st SERVE %',
#     '1st SERVE POINTS WON',
#     '2nd SERVE POINTS WON',
#     'TOTAL RETURN POINTS WON',
#     'TOTAL POINTS WON',
#     'DOUBLE FAULTS',
#     'ACES'
# ]

# # List of statistics to extract 
# stats_length = len(soup.find_all('td', class_='info_txt'))

# if stats_length == 8:
#     statistics.insert(3, 'BREAK POINTS WON')  # Insert 'BREAK POINTS WON' at the correct position



# # Dictionary to hold the statistics
# stat_data = {}

# # Extract and store statistics for both players
# for stat in statistics:
#     player1_stat, player2_stat = extract_statistic(stat)
#     stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
#     stat_data[f'{stat_name}_player1'] = player1_stat
#     stat_data[f'{stat_name}_player2'] = player2_stat

# # Convert the dictionary to a DataFrame
# df_stats = pd.DataFrame([stat_data])

# # Combine the DataFrames by column binding them
# df_combined = pd.concat([df_match_info, df_stats], axis=1)

# df_combined

# Extract Data from Multiple Links 

In [11]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [12]:

def extract_match_data(urls):
    data_list = []
    
    for url in urls:
        # Fetch HTML content from the URL
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract match information
        date = soup.find('td', class_='w50').text.strip().split(' ')[0]
        round_info = soup.find_all('td', class_='w50')[1].text.strip()
        player1 = soup.find_all('td', class_='w130')[0].text.strip()
        player2 = soup.find_all('td', class_='w130')[1].text.strip()
        score = soup.find('span', id='score').text.strip()
        tournament = soup.find('td', class_='w200').find('a').text.strip()
        
        # Extract statistics function
        def extract_statistic(statistic):
            row = soup.find('td', string=statistic).parent
            player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
            player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
            return player1_stat, player2_stat
        
            # List of statistics to extract
            statistics = [
                '1st SERVE %',
                '1st SERVE POINTS WON',
                '2nd SERVE POINTS WON',
                'TOTAL RETURN POINTS WON',
                'TOTAL POINTS WON',
                'DOUBLE FAULTS',
                'ACES'
            ]

            # List of statistics to extract 
            stats_length = len(soup.find_all('td', class_='info_txt'))

            if stats_length == 8:
                statistics.insert(3, 'BREAK POINTS WON')  # Insert 'BREAK POINTS WON' at the correct position

        
        # Extract statistics for original players
        
        stat_data = {}
        
        for stat in statistics:
            player1_stat, player2_stat = extract_statistic(stat)
            stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
            stat_data[f'{stat_name}_player1'] = player1_stat
            stat_data[f'{stat_name}_player2'] = player2_stat
        
        # Append match data to the list
        match_data = {
            'Date': date,
            'Round': round_info,
            'Player 1': player1,
            'Player 2': player2,
            'Score': score,
            'Tournament': tournament,
            **stat_data
        }
        data_list.append(match_data)
    
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data_list)
    
    # Sort DataFrame by Date in descending order
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%y')
    
    df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)
    
    return df


In [ ]:
extract_match_data(['https://www.tennislive.net/atp/match/bernard-tomic-VS-govind-nanda/m25-tulsa-2024/'])

### Accomodating for the player profile link

In [14]:
response = requests.get('https://www.tennislive.net/atp/match/rudy-quan-VS-shintaro-imai/little-rock-challenger-2024/')
soup = BeautifulSoup(response.content, 'html.parser')

### Modify Function
- https://www.tennislive.net/atp/match/rudy-quan-VS-jonas-pelle-hartenstein/m15-orange-park-2024/
- ^modify function for profiles with no statistics

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_match_data(urls):
    data_list = []
    
    for url in urls:
        # Fetch HTML content from the URL
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract match information
        date = soup.find('td', class_='w50').text.strip().split(' ')[0]
        round_info = soup.find_all('td', class_='w50')[1].text.strip()
        player1 = soup.find_all('td', class_='w130')[0].text.strip()
        player2 = soup.find_all('td', class_='w130')[1].text.strip()
        score = soup.find('span', id='score').text.strip()
        tournament = soup.find('td', class_='w200').find('a').text.strip()
        
        # Extract statistics function
        def extract_statistic(statistic):
            row = soup.find('td', string=statistic).parent
            player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
            player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
            return player1_stat, player2_stat
        
        # List of statistics to extract
        statistics = [
            '1st SERVE %',
            '1st SERVE POINTS WON',
            '2nd SERVE POINTS WON',
            'TOTAL RETURN POINTS WON',
            'TOTAL POINTS WON',
            'DOUBLE FAULTS',
            'ACES'
        ]

        # List of statistics to extract 
        stats_length = len(soup.find_all('td', class_='info_txt'))

        if stats_length == 8:
            statistics.insert(3, 'BREAK POINTS WON')  # Insert 'BREAK POINTS WON' at the correct position
        
        # Extract statistics for original players
        stat_data = {}
        table = soup.find_all('table', class_ = 'table_stats_match')

        if len(table) > 0 and table[0].find('td', class_='info_txt'):  # Process only if the table has data
            for stat in statistics:
                player1_stat, player2_stat = extract_statistic(stat)  # Extract stats for each player
                stat_name = (stat.lower()
                                .replace(' ', '_')
                                .replace('%', 'percentage')
                                .replace('/', '_')
                                .replace('(', '')
                                .replace(')', ''))

                # Store stats in dictionary
                stat_data[f'{stat_name}_player1'] = player1_stat
                stat_data[f'{stat_name}_player2'] = player2_stat
        else:
            # If the table is empty, set all stats to None
            for stat in statistics:
                stat_name = (stat.lower()
                                .replace(' ', '_')
                                .replace('%', 'percentage')
                                .replace('/', '_')
                                .replace('(', '')
                                .replace(')', ''))

                stat_data[f'{stat_name}_player1'] = None
                stat_data[f'{stat_name}_player2'] = None

        # Append match data to the list
        match_data = {
            'Date': date,
            'Round': round_info,
            'Player 1': player1,
            'Player 2': player2,
            'Score': score,
            'Tournament': tournament,
            **stat_data
        }
        data_list.append(match_data)
    
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data_list)
    
    # Sort DataFrame by Date in descending order
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%y')
    
    df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)
    
    return df

In [ ]:
# Example usage
urls = ['https://www.tennislive.net/atp/match/rudy-quan-VS-viktor-frydrych/us-open-juniors-2024/#google_vignette',
       'https://www.tennislive.net/atp/match/rudy-quan-VS-jonas-pelle-hartenstein/m15-orange-park-2024/',
       'https://www.tennislive.net/atp/match/rudy-quan-VS-viktor-frydrych/us-open-juniors-2024/']
df = extract_match_data(urls)
df

In [ ]:
extract_match_data(['https://www.tennislive.net/atp/match/rudy-quan-VS-jonas-pelle-hartenstein/m15-orange-park-2024/'])

In [ ]:
extract_match_data(['https://www.tennislive.net/atp/match/rudy-quan-VS-shintaro-imai/little-rock-challenger-2024/'])

In [ ]:
extract_match_data(['https://www.tennislive.net/atp/match/rudy-quan-VS-viktor-frydrych/us-open-juniors-2024/'])

### INPUT (playerName) and (links) HERE

In [ ]:
# Example usage:
# player_name = "Mikel Lopez Hernaez"

urls = ['https://www.tennislive.net/atp/match/rudy-quan-VS-jonas-pelle-hartenstein/m15-orange-park-2024/',
        'https://www.tennislive.net/atp/match/rudy-quan-VS-shintaro-imai/little-rock-challenger-2024/',
        'https://www.tennislive.net/atp/match/rudy-quan-VS-filip-peliwo/little-rock-challenger-2024/',
        'https://www.tennislive.net/atp/match/rudy-quan-VS-stefan-kozlov/little-rock-challenger-2024/',
        'https://www.tennislive.net/atp/match/rudy-quan-VS-andres-andrade/little-rock-challenger-2024/',
        'https://www.tennislive.net/atp/match/yuta-shimizu-VS-rudy-quan/little-rock-challenger-2024/'
]

# extract_match_data(player_name, urls)
extract_match_data(urls)

# page.status_code

# Accomodate for Player Profile Page

### Cj Notes

In [21]:
page = requests.get('https://www.tennislive.net/atp/ethan-quinn/')

In [22]:
soup = BeautifulSoup(page.text, 'html')

In [23]:
table = soup.find_all('table', class_ = 'table_pmatches')[1]

In [24]:
rows = table.find_all('td', class_='w50')

### Grab the Link
##### WARNING: This also grabs the links of the player profiles, we only want the match links
- use the fact that the match links are every 3rd index as per the previous cells

In [ ]:
match_links = []

for a in table.find_all('a', href=True):
    if "https://www.tennislive.net/atp/match/" in a['href']:
        match_links.append(a['href'])
        
        
match_links

In [ ]:
extract_match_data(match_links)

In [27]:
def player_profile(url):
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    
    table = soup.find_all('table', class_ = 'table_pmatches')[1]
    rows = table.find_all('td', class_='w50')
    
    
    match_links = []

    for a in table.find_all('a', href=True):
        if "https://www.tennislive.net/atp/match/" in a['href']:
            match_links.append(a['href'])
            
    return extract_match_data(match_links)


# (edit this function to accomodate for missing stats error)
#     - so copy and paste the function to edit it before this cell
    
#     output dataframe

In [ ]:
url = 'https://www.tennislive.net/atp/govind-nanda/'
player_profile(url)

In [ ]:
url = 'https://www.tennislive.net/atp/rudy-quan/'
player_profile(url)

player_data.to_csv(f'GovindNanda_profile.csv', index=False)

# Point by Point (PBP) Dataframe

In [31]:
page = requests.get('https://www.tennislive.net/atp/match/borna-gojo-VS-rudy-quan/sioux-falls-challenger-2024/')

In [32]:
soup = BeautifulSoup(page.text, 'html')

### Check Status Code
- 200 = success
- 404 = failed

In [ ]:
page.status_code

### Find HTML Tags that Have Point By Point (PBP) Data

The `table` tag with the class `table_stats_match` contains all the info needed for the point by point data.

In [34]:
table = soup.find_all('table', class_ = 'table_stats_match')
# table[1]

### First Set HTML

In [ ]:
set_1 = soup.find_all('table', class_='table_stats_match')[1].find_all('tr', class_=None)
set_1[0]

### Example of Getting Server of First Game

In [ ]:
' '.join(set_1[0].find('img')['alt'].split()[0:2])

### Example of Extracting PBP From First Game

In [ ]:
set_1[1].find('td')

Here we replace `[BP]` with signifies a "break-point" in the game with empty strings to keep the point data consistent. Then we can simply split the string into the points.

In [ ]:
set_1[1].find('td').text.replace('[BP]', '').split(', ')

### Example of Creating a PBP DataFrame from Set 1

In [39]:
# initialize lists to hold each of the designated values
server_names = []
point_scores = []
game_scores = []

# iterate over all the `tr` tags in set_1
# step over by 2 since each server and PBP data is paired together
for i in range(0, len(set_1), 2):
    # extract the data from the tr
    server_name = ' '.join(set_1[i].find('img')['alt'].split()[0:2])
    game_score = set_1[i].find('td', class_='mp_info_txt').text.strip()
    points = set_1[i + 1].find('td').text.replace('[BP]', '').split(', ')
    
    # create a row for each point in the game
    for point in points:
        server_names.append(server_name)
        point_scores.append(point)
        game_scores.append(game_score)


In [ ]:
# construct the dataframe 
df = pd.DataFrame({
    'point_score': point_scores,
    'server_name': server_names,
    'game_score': game_scores
})

# replace the last known score with 0-0 since the score
# doesn't update between sets
df = df.replace(df['game_score'].iloc[-1], '0-0')
df

### Find and Denote Each Player to Correctly Swap the Scores

We do this because the PBP is always in the order of `player1`-`player2`. To make the dataframe easier to understand, we swap the point and game scores when the server is `player2` as the server's points are read first.

In [ ]:
table = soup.find('table', class_ = 'table_pmatches')

player1 = soup.find_all('td', class_='w130')[0].text.strip()
player2 = soup.find_all('td', class_='w130')[1].text.strip()

player1, player2

In [ ]:
df.iloc[3]

In [ ]:
'-'.join(df['point_score'][3].split('-')[::-1])

In [44]:
df['point_score'] = df.apply(lambda x : '-'.join(x['point_score'].split('-')[::-1]) if x['server_name'] == player2 else x['point_score'], axis=1)
df['game_score'] = df.apply(lambda x : '-'.join(x['game_score'].split('-')[::-1]) if x['server_name'] == player2 else x['game_score'], axis=1)

In [ ]:
df

### Generalize the Scraping to Apply to All Sets

We'll add a new column to keep track of the set number as well to help with readability in the dataframe.

In [93]:
# 2 set example
page = requests.get('https://www.tennislive.net/atp/match/borna-gojo-VS-rudy-quan/sioux-falls-challenger-2024/')

In [94]:
soup = BeautifulSoup(page.text, 'html')

In [95]:
def extract_point_by_point(url):
    # get HTML from url and convert to BeautifulSoup
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    
    # find player names
    table = soup.find('table', class_ = 'table_pmatches')
    player1 = soup.find_all('td', class_='w130')[0].text.strip()
    player2 = soup.find_all('td', class_='w130')[1].text.strip()
    
    # get all set data
    content = soup.find_all('table', class_='table_stats_match')[1:]

    # initialize list to hold data for all sets
    set_dfs = []

    for j in range(len(content)):
        # initialize lists to hold each of the designated values
        server_names = []
        point_scores = []
        game_scores = []
        set_num = []
        is_break_points = []

        # get all table data for the current set
        current_set = content[j].find_all('tr', class_=None)

        # determine starting tag by how the table is formatted
        start = 0
        if not current_set[1].find('td', class_='mp_15'):
            start = 1

        if len(current_set) % 2 == 0:
            length_current_set = len(current_set) - 1
        else:
            length_current_set = len(current_set)

        # iterate over all the `tr` tags in set_1
        # step over by 2 since each server and PBP data is paired together
        for i in range(start, length_current_set, 2):
            # extract the data from the tr
            server_name = ' '.join(current_set[i].find('img')['alt'].split()[0:-1])
            game_score = current_set[i].find('td', class_='mp_info_txt').text.strip()
            points = current_set[i + 1].find('td').text.split(', ')

            # create a new row for each point
            for point in points:
                
                if '[BP]' in point:
                    is_break_point = True
                    point = point.replace('[BP]', '').strip()
                else:
                    is_break_point = False

                server_names.append(server_name)
                point_scores.append(point)
                game_scores.append(game_score)
                set_num.append(j + 1)
                is_break_points.append(is_break_point)

        # construct the dataframe
        df = pd.DataFrame({
            'pointScore': point_scores,
            'serverName': server_names,
            'gameScore': game_scores,
            'setNum': set_num,
            'isBreakPoint': is_break_points
        })

        last_game_score = df['gameScore'].iloc[-1]

        if '6' in last_game_score and (last_game_score != '6-6' and last_game_score != '6-5' and last_game_score != '5-6'):
            df.loc[df['gameScore'] == df['gameScore'].iloc[-1], 'setNum'] += 1
            df.loc[df['gameScore'] == df['gameScore'].iloc[-1], 'gameScore'] = '0-0' 

            # switch server names when in tiebreaker        
        if '6-6' in last_game_score or '0-0' in last_game_score:
            tiebreaker_df = df[df['gameScore'] == last_game_score]
            tiebreaker_rows = df.shape[0] - tiebreaker_df.shape[0]
            server = tiebreaker_df['serverName'].iloc[0]

            if player1 != server:
                server = player1

            for i in range(1, len(tiebreaker_df), 4):
                df.loc[tiebreaker_rows + i:tiebreaker_rows + i + 1, 'serverName'] = server
                
        set_dfs.append(df)
        
    df = pd.concat(set_dfs).reset_index(drop=True)    

    # flip point_score and game_score of player2
    df['pointScore'] = df.apply(lambda x : '-'.join(x['pointScore'].split('-')[::-1]) if x['serverName'] == player2 else x['pointScore'], axis=1)
    df['gameScore'] = df.apply(lambda x : '-'.join(x['gameScore'].split('-')[::-1]) if x['serverName'] == player2 else x['gameScore'], axis=1)
    
    return df

In [ ]:
url = 'https://www.tennislive.net/atp/match/borna-gojo-VS-rudy-quan/sioux-falls-challenger-2024/'
extract_point_by_point(url)

In [ ]:
url = 'https://www.tennislive.net/atp/match/govind-nanda-VS-karue-sell/m15-lakewood-2024/'
extract_point_by_point(url)

### More Examples

Some other interesting match urls are tested below to test functionality of varying sets and number of tiebreaks.

In [ ]:
url = 'https://www.tennislive.net/atp/match/yuta-shimizu-VS-rudy-quan/little-rock-challenger-2024/'
extract_point_by_point(url).head(20)

In [ ]:
url = 'https://www.tennislive.net/atp/match/jorge-plans-VS-mikel-lopez-hernaez/m25-bakio-2024/'
extract_point_by_point(url)

In [ ]:
url = 'https://www.tennislive.net/atp/match/jorge-plans-VS-mikel-lopez-hernaez/m25-bakio-2024/'
extract_point_by_point(url)

In [ ]:
url = 'https://www.tennislive.net/atp/match/mitchell-krueger-VS-abedallah-shelbayh/little-rock-challenger-2024/'
extract_point_by_point(url)

In [ ]:
url = 'https://www.tennislive.net/atp/match/ethan-quinn-VS-rudy-quan/fairfield-challenger-2024/'
extract_point_by_point(url)

# Input link and modify data frame to be formatted correctly

In [31]:
url = 'https://www.tennislive.net/atp/match/yuta-shimizu-VS-rudy-quan/little-rock-challenger-2024/'
df = extract_point_by_point(url)

In [ ]:
df[df['setNum'] == 1]

## Add Columns

### Add pointNumber column

In [33]:
df.insert(0, 'pointNumber', range(1, len(df) + 1))

### Add gameNumber column

In [34]:
def game_score(game):
    return sum(map(int,('').join(game.split('-')))) + 1

df['gameNumber'] = df['gameScore'].apply(game_score)

### Add player1Name and player2Name columns

In [35]:
serverNames = df['serverName'].unique()

player1 = serverNames[0]
player2 = serverNames[1]

df['player1Name'] = player1
df['player2Name'] = player2

### Add returnName column

In [36]:
def switch_names(name):
    if name != player1:
        return player1
    else :
        return player2

df['returnerName'] = df['serverName'].apply(switch_names)

### Add tiebreakeScore column

In [37]:
# Transfer values from pointScore to tiebreakScore where gameScore is '6-6'
if 'tiebreakScore' not in df.columns:
    df['tiebreakScore'] = np.nan

for i in df['setNum'].unique():
    current_set = df[df['setNum'] == i]

    last_game_score = current_set['gameScore'].iloc[-1]

    if '6-6' in last_game_score or '0-0' in last_game_score:
        df.loc[(df['setNum'] == i) & (df['gameScore'] == last_game_score), 'tiebreakScore'] = df['pointScore']
        
        # Set the pointScore to NaN where tiebreakScore is not NaN
        df.loc[pd.notna(df['tiebreakScore']), 'pointScore'] = np.nan

### Add setScore column
#### WARNING: Manually input the setScores for desired player

In [38]:
set_list = ['0-0', '1-0']

def set_setScores(x):
    
    if 1 <= x <= len(set_list):
        return set_list[x - 1]
        
df['setScore'] = df['setNum'].apply(set_setScores)

In [ ]:
df

### Add Timestamp columns

In [40]:
df['Position'] = ''
df['pointEndPosition'] = ''
df['Duration'] = ''

### Add Team columns
#### WARNING: Manually set Team Names

In [41]:
df['clientTeam'] = 'UCLA'
df['opponentTeam'] = ''

### Add name column

In [42]:
df['Name'] = df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)

## Specify Order

In [43]:
df = df[['Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name', 'player2Name', 
         'pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'serverName', 'returnerName',
        'Position', 'pointEndPosition', 'Duration', 'isBreakPoint', 'clientTeam', 'opponentTeam']]


In [ ]:
df.head(5)

# Add in Timestamps

In [ ]:
# # Put timestamps file here
# timestamp = pd.read_csv("timestamps_9uYpCdMy4eA.csv")

In [ ]:
# if timestamp.shape[0] != point_df.shape[0]:
#     raise ValueError("Error: The number of rows in timestamp and point_df are not the same.")
# else:
#     # Assign values to point_df
#     point_df['Position'] = timestamp['pointStartTime'].values
#     point_df['pointEndPosition'] = timestamp['endStartTime'].values
#     print("\u2713 Check passed")

# Output the Point Visuals csv

In [ ]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
df.to_csv(f'Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)